# Analyze Results

In [ ]:
import os.path
import pandas as pd

from dbispipeline.analytics import get_results_as_dataframe
from hit_prediction_code.analytics import add_approach_to_df
from hit_prediction_code.analytics import add_cv_epoch_evaluator_outcome_to_df

results = get_results_as_dataframe(project_name='hit-prediction-ismir2020', table_name='hit_prediction', filter_git_dirty=False)
results = results[results['date'] > '2021-02-02']

add_approach_to_df(results)
add_cv_epoch_evaluator_outcome_to_df(results)

In [ ]:
import numpy as np

from matplotlib import pyplot
import seaborn as sns

results = results.tail(1)
mean_results = results['mean'].iloc[0]
# print(mean_results.head(14).to_markdown())

cm_results = mean_results.loc['confusion_matrix']

cm_2 = np.array(cm_results[2])

tp = np.sum(cm_2[:-1,:-1])
tn = cm_2[-1,-1]

p = np.sum(cm_2[:-1])
n = np.sum(cm_2[-1])

hit_nonhit_accuracy_25 = (tp + tn) / (p + n)
print(hit_nonhit_accuracy_25)

# y_pred_sums = np.sum(cm_2, axis=0) * range(1, 102)
# y_true_sums = np.sum(cm_2, axis=1) * range(1, 102)

# print(np.sum(y_pred_sums) / np.sum(cm_2), np.sum(y_true_sums) / np.sum(cm_2))

# print(np.average(, weights=range(1,102)), np.average(np.sum(cm_results[2], axis=1), weights=range(1,102)))

target_counts = np.sum(cm_results[0], axis=1) + 1

divisor = np.broadcast_to(target_counts + 1, (len(target_counts), (len(target_counts)))).transpose()

normalized_cm = []

for cm in cm_results:
    cm = np.array(cm)
    normalized_cm.append(cm / divisor)
    
for i in range(0, int(len(normalized_cm)), 1):
    pyplot.subplots(figsize=(20,15))
    plot = sns.heatmap(normalized_cm[i])
    plot.set_title('%s Epoches' % mean_results.columns[i])
    pyplot.show()
#     pyplot.savefig('../results/cm_norm_25_200.pdf')